**Helper notebook**

In [ ]:
%run nb_helper

**Parameters**

In [ ]:
pSourceWorkspaceId = ""
pProjectPath = "" # path to the extraction in your cicd lakehouse

**Access token and base url**

In [ ]:
vScope = "https://analysis.windows.net/powerbi/api"
vAccessToken  = mssparkutils.credentials.getToken(vScope)
vBaseUrl = f"https://api.fabric.microsoft.com/v1/"
vHeaders = {'Authorization': f'Bearer {vAccessToken}'}

**Resolve source workspace ids**

In [ ]:
vSourceWorkspaceName = fabric.resolve_workspace_name(pSourceWorkspaceId)

**List source lakehouses**

In [ ]:
df_source_lakehouses = labs.list_lakehouses(workspace=vSourceWorkspaceName)

**Identify current access roles**

In [ ]:
try:

    df_onelake_access = pd.DataFrame()

    # get a token for storage
    vOnelakeHeaders = {"authorization": f"bearer {mssparkutils.credentials.getToken('storage')}"}

    # iterate over the lakehouses
    for index, row in df_source_lakehouses.iterrows():

        # set the lakehouse name and id
        vLakehouseName = row['Lakehouse Name']
        vLakehouseSourceId = row['Lakehouse ID']
        vLakehouseTargetId = labs.resolve_lakehouse_id(lakehouse=vLakehouseName, workspace=vSourceWorkspaceName)


        # 3. extract onelake access
        vExtractionType = "onelake_access"
        vShortcutUrl = f"workspaces/{pSourceWorkspaceId}/items/{vLakehouseSourceId}/dataAccessRoles"
        vUrl = vBaseUrl + vShortcutUrl
        print(f"extracting onelake access for lakehouse {vLakehouseName}")  

        # create the api global dataframe for shortcuts
        api_call_global_dataframe = pd.DataFrame()

        try:
                
            # make the api call
            api_call_main(vUrl, vHeaders, 'yes', vExtractionType)

            api_call_global_dataframe['lakehouse'] = vLakehouseName


            # concat to the correspondant dataframe
            df_onelake_access = pd.concat([df_onelake_access, api_call_global_dataframe], ignore_index=True)

            # logging
            vMessage = f"extracting onelake access for lakehouse {vLakehouseName} succeeded"
            print(vMessage)

        except Exception as e:
            print(str(e))

except Exception as e:
    print(str(e))

**Extract rules, entra members and item members**

In [ ]:
# 2. prepare the inputs for the creation
df_role_rules = flatten_nested_json_df(df_onelake_access[['id', 'decisionRules']].explode('decisionRules').dropna())
condition_1 = (df_role_rules["decisionRules.permission.attributeName"] == "Action") & (df_role_rules["decisionRules.permission.attributeValueIncludedIn"] != "Read")
df_role_rules_1 = df_role_rules[~condition_1]
condition_2 = (df_role_rules_1["decisionRules.permission.attributeName"] == "Path") & (df_role_rules_1["decisionRules.permission.attributeValueIncludedIn"] == "Read")
df_role_rules_2 = df_role_rules_1[~condition_2]
df_role_rules = df_role_rules_2
df_entra_members = flatten_nested_json_df(df_onelake_access[['id', 'members.microsoftEntraMembers']].explode('members.microsoftEntraMembers').dropna())
df_item_members = flatten_nested_json_df(df_onelake_access[['id', 'members.fabricItemMembers']].explode('members.fabricItemMembers').dropna())

**Generate the onelake_roles.json file**

In [ ]:
df_onelake_roles = df_onelake_access.reset_index()[["index", "name", "id", "lakehouse"]]
onelake_roles_json = df_onelake_roles.apply(lambda row: {col: row[col] for col in df_onelake_roles.columns if not pd.isna(row[col])}, axis=1).tolist()

fileName = "onelake_roles.json"
filePath = f'Files/{pProjectPath}/{fileName}'

with open(f'/lakehouse/default/{filePath}','w') as f:
    json.dump(onelake_roles_json,f)

**Generate the onelake_rules.json file**

In [ ]:
onelake_rules_json = df_role_rules.apply(lambda row: {col: row[col] for col in df_role_rules.columns if not pd.isna(row[col])}, axis=1).tolist()

fileName = "onelake_rules.json"
filePath = f'Files/{pProjectPath}/{fileName}'

with open(f'/lakehouse/default/{filePath}','w') as f:
    json.dump(onelake_rules_json,f)

**Generate the onelake_entra_members.json file**

In [ ]:
onelake_entra_members_json = df_entra_members.apply(lambda row: {col: row[col] for col in df_entra_members.columns if not pd.isna(row[col])}, axis=1).tolist()

fileName = "onelake_entra_members.json"
filePath = f'Files/{pProjectPath}/{fileName}'

with open(f'/lakehouse/default/{filePath}','w') as f:
    json.dump(onelake_entra_members_json,f)

**Generate the onelake_item_members.json file**

In [ ]:
onelake_item_members_json = df_item_members.apply(lambda row: {col: row[col] for col in df_item_members.columns if not pd.isna(row[col])}, axis=1).tolist()

fileName = "onelake_item_members.json"
filePath = f'Files/{pProjectPath}/{fileName}'

with open(f'/lakehouse/default/{filePath}','w') as f:
    json.dump(onelake_item_members_json,f)

In [ ]:
pOnelakeRoles = json.dumps(onelake_roles_json, separators=(",", ":"))
pOnelakeRules = json.dumps(onelake_rules_json, separators=(",", ":"))
pOnelakeEntraMembers = json.dumps(onelake_entra_members_json, separators=(",", ":"))
pOnelakeItemMembers = json.dumps(onelake_item_members_json, separators=(",", ":"))

In [ ]:
# print(pOnelakeRoles)
# print(pOnelakeRules)
# print(pOnelakeEntraMembers)
# print(pOnelakeItemMembers)